In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols # linear ANOVA
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import f_oneway
import statsmodels.formula.api as smf

/Users/mahdie/opt/anaconda3/envs/physioai/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


##### Data loading

In [2]:
maxwalk = pd.read_csv("data/maxwalk.csv")
maxwalk.shape

(1361, 11)

In [3]:
maxwalk

,mergeid,age,bmi,female,educational_level,OA_conserv,Hypertension,Diabetes,maxgrip,walking,disease_category
0,AT-078466-02,77.0,26.562500,1.0,1.0,0.0,0.0,0.0,23.0,2.495,No disease
1,AT-111399-02,80.0,20.395421,1.0,1.0,0.0,0.0,0.0,22.0,2.265,No disease
2,AT-567061-01,77.0,29.757585,0.0,1.0,0.0,0.0,0.0,39.0,2.390,No disease
3,AT-934728-02,76.0,20.438166,1.0,1.0,0.0,1.0,0.0,23.0,4.985,Only HT
4,AT-971813-02,77.0,29.060607,0.0,1.0,0.0,0.0,0.0,41.0,4.095,No disease
...,...,...,...,...,...,...,...,...,...,...,...
1356,SE-939741-02,77.0,28.066424,0.0,0.0,0.0,1.0,0.0,39.0,4.360,Only HT
1357,SE-965987-02,76.0,31.644286,1.0,0.0,0.0,1.0,0.0,33.0,2.905,Only HT
1358,SE-975106-01,80.0,32.596372,1.0,0.0,0.0,1.0,1.0,26.0,3.140,HT and Diab
1359,SE-988222-01,80.0,24.464602,1.0,0.0,0.0,0.0,0.0,27.0,2.330,No disease


## statistical

##### Statistics on df_features

- **0**: No disease  
- **1**:  
  - Diabetes only  
  - Hypertension only  
  - OA only  
- **2**:  
  - Diabetes + Hypertension  
  - Diabetes + OA  
  - Hypertension + OA  
- **3**: Diabetes + Hypertension + OA 


In [4]:
maxwalk['disease_category'] = 'No disease'  # Default group
maxwalk.loc[(maxwalk['OA_conserv'] == 1) & (maxwalk['Hypertension'] == 0) & (maxwalk['Diabetes'] == 0), 'disease_category'] = 'Only OA'
maxwalk.loc[(maxwalk['OA_conserv'] == 0) & (maxwalk['Hypertension'] == 1) & (maxwalk['Diabetes'] == 0), 'disease_category'] = 'Only HT'
maxwalk.loc[(maxwalk['OA_conserv'] == 0) & (maxwalk['Hypertension'] == 0) & (maxwalk['Diabetes'] == 1), 'disease_category'] = 'Only Diabetes'
maxwalk.loc[(maxwalk['OA_conserv'] == 1) & (maxwalk['Hypertension'] == 1) & (maxwalk['Diabetes'] == 0), 'disease_category'] = 'OA and HT'
maxwalk.loc[(maxwalk['OA_conserv'] == 1) & (maxwalk['Hypertension'] == 0) & (maxwalk['Diabetes'] == 1), 'disease_category'] = 'OA and Diab'
maxwalk.loc[(maxwalk['OA_conserv'] == 0) & (maxwalk['Hypertension'] == 1) & (maxwalk['Diabetes'] == 1), 'disease_category'] = 'HT and Diab'
maxwalk.loc[(maxwalk['OA_conserv'] == 1) & (maxwalk['Hypertension'] == 1) & (maxwalk['Diabetes'] == 1), 'disease_category'] = 'All three diseases'


### Maxgrip

#### No confounding

##### Step1: One-Way ANOVA

Checks if any group differs

In [5]:
groups = maxwalk['disease_category'].unique()
grip_samples = [maxwalk[maxwalk['disease_category'] == group]['maxgrip'].dropna() for group in groups]

# Run ANOVA
anova_grip = f_oneway(*grip_samples)
print("ANOVA for Grip Strength")
print(f"F-statistic: {anova_grip.statistic:.3f}, p-value: {anova_grip.pvalue:.5f}")


ANOVA for Grip Strength
F-statistic: 5.940, p-value: 0.00000


##### Step2: Tukey HSD Post-hoc Comparison

Identifies which groups differ

In [6]:
df_grip = maxwalk[['maxgrip', 'disease_category']].dropna()

# Run Tukey HSD
tukey_grip = pairwise_tukeyhsd(endog=df_grip['maxgrip'],
                               groups=df_grip['disease_category'],
                               alpha=0.05)
print("Turkey Maxgrip in age 75 or above 75")
print(tukey_grip.summary())


Turkey Maxgrip in age 75 or above 75
          Multiple Comparison of Means - Tukey HSD, FWER=0.05          
      group1           group2    meandiff p-adj   lower   upper  reject
-----------------------------------------------------------------------
All three diseases   HT and Diab   1.6842 0.9845 -3.8932  7.2616  False
All three diseases    No disease   4.9136 0.0367  0.1629  9.6643   True
All three diseases   OA and Diab   1.9759 0.9924 -5.3903  9.3421  False
All three diseases     OA and HT   2.2056 0.9101 -3.0695  7.4806  False
All three diseases Only Diabetes   2.4943 0.8761 -3.0831  8.0717  False
All three diseases       Only HT   3.1868  0.472 -1.6143  7.9879  False
All three diseases       Only OA    -0.09    1.0 -5.3284  5.1484  False
       HT and Diab    No disease   3.2294 0.0796 -0.1865  6.6453  False
       HT and Diab   OA and Diab   0.2917    1.0 -6.2931  6.8765  False
       HT and Diab     OA and HT   0.5214 0.9999 -3.5926  4.6354  False
       HT and Diab Only Dia

##### Number of samples in each group

In [7]:
group_counts = maxwalk[maxwalk['maxgrip'].notna()].groupby('disease_category').size()
print(group_counts)

disease_category
All three diseases     38
HT and Diab            79
No disease            510
OA and Diab            24
OA and HT             117
Only Diabetes          79
Only HT               390
Only OA               124
dtype: int64


##### Step3: Code to Compute Cohen's d and 95% CI

In [8]:
comparisons = [
    ("No disease", "Only OA"),
    ("No disease", "Only HT"),
    ("No disease", "Only Diabetes"),
    ("No disease", "HT and Diab"),
    ("No disease", "OA and Diab"),
    ("No disease", "OA and HT"),
    ("No disease", "All three diseases")
]

results = []

for g1, g2 in comparisons:
   
    group1 = maxwalk[(maxwalk['disease_category'] == g1)]['maxgrip'].dropna()
    group2 = maxwalk[(maxwalk['disease_category'] == g2)]['maxgrip'].dropna()

    mean1, mean2 = group1.mean(), group2.mean()
    std1, std2 = group1.std(), group2.std()
    n1, n2 = len(group1), len(group2)


    pooled_sd = np.sqrt(((n1 - 1)*std1**2 + (n2 - 1)*std2**2) / (n1 + n2 - 2))

    cohen_d = (mean1 - mean2) / pooled_sd

    diff = mean1 - mean2
    se_diff = np.sqrt(std1**2 / n1 + std2**2 / n2)
    ci_low, ci_high = diff - 1.96 * se_diff, diff + 1.96 * se_diff

    results.append({
        'Comparison': f"{g1} vs {g2}",
        'Mean1': round(mean1, 2),
        'Mean2': round(mean2, 2),
        'Cohen_d': round(cohen_d, 3),
        '95% CI Lower': round(ci_low, 3),
        '95% CI Upper': round(ci_high, 3),
        'n1': n1,
        'n2': n2
    })

effect_df = pd.DataFrame(results)
effect_df


,Comparison,Mean1,Mean2,Cohen_d,95% CI Lower,95% CI Upper,n1,n2
0,No disease vs Only OA,29.23,24.23,0.538,3.201,6.806,510,124
1,No disease vs Only HT,29.23,27.50,0.185,0.492,2.961,510,390
2,No disease vs Only Diabetes,29.23,26.81,0.262,0.360,4.478,510,79
3,No disease vs HT and Diab,29.23,26.00,0.352,1.269,5.190,510,79
4,No disease vs OA and Diab,29.23,26.29,0.317,0.001,5.875,510,24
5,No disease vs OA and HT,29.23,26.52,0.281,0.595,4.822,510,117
6,No disease vs All three diseases,29.23,24.32,0.528,2.028,7.799,510,38


#### With confounding: age, BMI, female, educational_level:

Run OLS model (adjusted for age, bmi, female)

In [9]:
maxwalk['disease_category'] = pd.Categorical(
    maxwalk['disease_category'],
    categories=[
        'No disease', 'Only OA', 'Only HT', 'Only Diabetes',
        'HT and Diab', 'OA and Diab', 'OA and HT', 'All three diseases'
    ],
    ordered=False
)


model = smf.ols('maxgrip ~ C(disease_category) + age + bmi + female + educational_level', data=maxwalk).fit()


results_df = pd.DataFrame({
    'Disease group': model.params.index,
    'Coefficient (Adj. Mean Diff)': model.params.values,
    '95% CI Lower': model.conf_int().iloc[:, 0],
    '95% CI Upper': model.conf_int().iloc[:, 1],
    'p-value': model.pvalues.values
})


results_df = results_df[results_df['Disease group'].str.contains('C\(disease_category\)')].copy()


results_df['Disease group'] = results_df['Disease group'].str.replace(r'C\(disease_category\)\[T\.', '', regex=True).str.rstrip(']')


results_df['Coefficient (Adj. Mean Diff)'] = results_df['Coefficient (Adj. Mean Diff)'].round(3)
results_df['95% CI Lower'] = results_df['95% CI Lower'].round(3)
results_df['95% CI Upper'] = results_df['95% CI Upper'].round(3)
results_df['p-value'] = results_df['p-value'].round(4)

results_df['Significant'] = results_df['p-value'].apply(lambda p: 'Yes' if p < 0.05 else 'No')

print("OLS results for maxgrip in age 75 or above 75 (No disease as reference):")
results_df.reset_index(drop=True)



OLS results for maxgrip in age 75 or above 75 (No disease as reference):


,Disease group,Coefficient (Adj. Mean Diff),95% CI Lower,95% CI Upper,p-value,Significant
0,Only OA,-3.076,-4.419,-1.734,0.0000,Yes
1,Only HT,-0.507,-1.410,0.396,0.2711,No
2,Only Diabetes,-2.222,-3.840,-0.603,0.0072,Yes
3,HT and Diab,-1.461,-3.087,0.164,0.0779,No
4,OA and Diab,-4.047,-6.837,-1.257,0.0045,Yes
5,OA and HT,-1.217,-2.600,0.166,0.0845,No
6,All three diseases,-2.902,-5.176,-0.627,0.0124,Yes


##### Interaction Effects: Does OA Get Worse With Comorbidities?

In [10]:
model = smf.ols('maxgrip ~ C(disease_category) + age + bmi + female + educational_level + C(OA_conserv)*C(Diabetes) + C(OA_conserv)*C(Hypertension)', 
                data=maxwalk).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                maxgrip   R-squared:                       0.486
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     115.7
Date:                Fri, 27 Jun 2025   Prob (F-statistic):          8.10e-186
Time:                        11:07:09   Log-Likelihood:                -4531.5
No. Observations:                1361   AIC:                             9087.
Df Residuals:                    1349   BIC:                             9150.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

##### Logistic Regression: Predicting "High Risk" Patients

In [11]:
maxwalk['low_grip'] = (maxwalk['maxgrip'] < maxwalk['maxgrip'].quantile(0.25)).astype(int)

X = maxwalk[['age', 'bmi', 'female', 'educational_level','OA_conserv', 'Diabetes', 'Hypertension']]
y = maxwalk['low_grip']

logit_model = sm.Logit(y, sm.add_constant(X)).fit()
print(logit_model.summary())


Optimization terminated successfully.
         Current function value: 0.393748
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               low_grip   No. Observations:                 1361
Model:                          Logit   Df Residuals:                     1353
Method:                           MLE   Df Model:                            7
Date:                Fri, 27 Jun 2025   Pseudo R-squ.:                  0.2046
Time:                        11:07:09   Log-Likelihood:                -535.89
converged:                       True   LL-Null:                       -673.78
Covariance Type:            nonrobust   LLR p-value:                 8.959e-56
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               -12.4704      1.644     -7.587      0.000     -15.692      -9.249
age     

### Walking

#### No confounding

##### Step1: One-Way ANOVA

Checks if any group differs

In [12]:
# ANOVA for walking speed
walking_samples = [maxwalk[maxwalk['disease_category'] == group]['walking'].dropna() for group in groups]

anova_walking = f_oneway(*walking_samples)
print("\nANOVA for walking speed")
print(f"F-statistic: {anova_walking.statistic:.3f}, p-value: {anova_walking.pvalue:.5f}")



ANOVA for walking speed
F-statistic: 1.565, p-value: 0.14174


##### Step2: Tukey HSD Post-hoc Comparison

Identifies which groups differ

In [13]:
# Tukey HSD for Walking speed
df_walking = maxwalk[['walking', 'disease_category']].dropna()

tukey_walking = pairwise_tukeyhsd(endog=df_walking['walking'],
                                groups=df_walking['disease_category'],
                                alpha=0.05)
print("Turkey Walking Speed in age 75 or above 75")
print(tukey_walking.summary())


Turkey Walking Speed in age 75 or above 75
         Multiple Comparison of Means - Tukey HSD, FWER=0.05          
      group1           group2    meandiff p-adj   lower  upper  reject
----------------------------------------------------------------------
All three diseases   HT and Diab   0.0286    1.0 -2.2812 2.3385  False
All three diseases    No disease   -0.791  0.926 -2.7585 1.1765  False
All three diseases   OA and Diab   0.1821    1.0 -2.8685 3.2328  False
All three diseases     OA and HT  -0.4238  0.999 -2.6084 1.7608  False
All three diseases Only Diabetes    0.202    1.0 -2.1079 2.5118  False
All three diseases       Only HT  -0.7978 0.9267 -2.7862 1.1905  False
All three diseases       Only OA  -0.2368    1.0 -2.4063 1.9326  False
       HT and Diab    No disease  -0.8196 0.6482 -2.2343  0.595  False
       HT and Diab   OA and Diab   0.1535    1.0 -2.5735 2.8806  False
       HT and Diab     OA and HT  -0.4524 0.9928 -2.1562 1.2514  False
       HT and Diab Only Diabetes  

##### Number of samples in each group

In [14]:
group_counts = maxwalk[maxwalk['walking'].notna()].groupby('disease_category').size()
print(group_counts)

disease_category
No disease            510
Only OA               124
Only HT               390
Only Diabetes          79
HT and Diab            79
OA and Diab            24
OA and HT             117
All three diseases     38
dtype: int64


/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_89500/3308135134.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_counts = maxwalk[maxwalk['walking'].notna()].groupby('disease_category').size()


##### Step3: Compute Cohen's d and 95% CI

In [15]:
comparisons = [
    ("No disease", "Only OA"),
    ("No disease", "Only HT"),
    ("No disease", "Only Diabetes"),
    ("No disease", "HT and Diab"),
    ("No disease", "OA and Diab"),
    ("No disease", "OA and HT"),
    ("No disease", "All three diseases")
]

results = []

for g1, g2 in comparisons:
    
    group1 = maxwalk[(maxwalk['disease_category'] == g1)]['walking'].dropna()
    group2 = maxwalk[(maxwalk['disease_category'] == g2)]['walking'].dropna()


    mean1, mean2 = group1.mean(), group2.mean()
    std1, std2 = group1.std(), group2.std()
    n1, n2 = len(group1), len(group2)

    pooled_sd = np.sqrt(((n1 - 1)*std1**2 + (n2 - 1)*std2**2) / (n1 + n2 - 2))


    cohen_d = (mean1 - mean2) / pooled_sd

   
    diff = mean1 - mean2
    se_diff = np.sqrt(std1**2 / n1 + std2**2 / n2)
    ci_low, ci_high = diff - 1.96 * se_diff, diff + 1.96 * se_diff

    results.append({
        'Comparison': f"{g1} vs {g2}",
        'Mean1': round(mean1, 2),
        'Mean2': round(mean2, 2),
        'Cohen_d': round(cohen_d, 3),
        '95% CI Lower': round(ci_low, 3),
        '95% CI Upper': round(ci_high, 3),
        'n1': n1,
        'n2': n2
    })

effect_df = pd.DataFrame(results)
effect_df


,Comparison,Mean1,Mean2,Cohen_d,95% CI Lower,95% CI Upper,n1,n2
0,No disease vs Only OA,4.79,5.34,-0.147,-1.300,0.191,510,124
1,No disease vs Only HT,4.79,4.78,0.002,-0.473,0.486,510,390
2,No disease vs Only Diabetes,4.79,5.78,-0.248,-2.218,0.232,510,79
3,No disease vs HT and Diab,4.79,5.61,-0.212,-1.878,0.239,510,79
4,No disease vs OA and Diab,4.79,5.76,-0.260,-2.397,0.451,510,24
5,No disease vs OA and HT,4.79,5.16,-0.097,-1.139,0.405,510,117
6,No disease vs All three diseases,4.79,5.58,-0.213,-1.859,0.277,510,38


#### With confounding: age, BMI, female, educational_level:

Run OLS model (adjusted for age, bmi, female)

In [16]:
maxwalk['disease_category'] = pd.Categorical(
    maxwalk['disease_category'],
    categories=[
        'No disease', 'Only OA', 'Only HT', 'Only Diabetes',
        'HT and Diab', 'OA and Diab', 'OA and HT', 'All three diseases'
    ],
    ordered=False
)


model = smf.ols('walking ~ C(disease_category) + age + bmi + female + educational_level', data=maxwalk).fit()


results_df = pd.DataFrame({
    'Disease group': model.params.index,
    'Coefficient (Adj. Mean Diff)': model.params.values,
    '95% CI Lower': model.conf_int().iloc[:, 0],
    '95% CI Upper': model.conf_int().iloc[:, 1],
    'p-value': model.pvalues.values
})


results_df = results_df[results_df['Disease group'].str.contains('C\(disease_category\)')].copy()


results_df['Disease group'] = results_df['Disease group'].str.replace(r'C\(disease_category\)\[T\.', '', regex=True).str.rstrip(']')


results_df['Coefficient (Adj. Mean Diff)'] = results_df['Coefficient (Adj. Mean Diff)'].round(3)
results_df['95% CI Lower'] = results_df['95% CI Lower'].round(3)
results_df['95% CI Upper'] = results_df['95% CI Upper'].round(3)
results_df['p-value'] = results_df['p-value'].round(4)
results_df['Significant'] = results_df['p-value'].apply(lambda p: 'Yes' if p < 0.05 else 'No')

print("OLS results for walking speed in age 75 or above (No disease as reference):")
results_df.reset_index(drop=True)


OLS results for walking speed in age 75 or above (No disease as reference):


,Disease group,Coefficient (Adj. Mean Diff),95% CI Lower,95% CI Upper,p-value,Significant
0,Only OA,0.305,-0.446,1.056,0.4261,No
1,Only HT,-0.141,-0.646,0.365,0.5848,No
2,Only Diabetes,0.842,-0.063,1.748,0.0683,No
3,HT and Diab,0.561,-0.349,1.470,0.2267,No
4,OA and Diab,0.905,-0.656,2.467,0.2555,No
5,OA and HT,0.069,-0.705,0.843,0.8618,No
6,All three diseases,0.466,-0.807,1.739,0.4726,No


In [17]:
group_counts = maxwalk[maxwalk['walking'].notna()].groupby('disease_category').size()
print(group_counts)

disease_category
No disease            510
Only OA               124
Only HT               390
Only Diabetes          79
HT and Diab            79
OA and Diab            24
OA and HT             117
All three diseases     38
dtype: int64


/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_89500/3308135134.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_counts = maxwalk[maxwalk['walking'].notna()].groupby('disease_category').size()


##### Interaction Effects: Does OA Get Worse With Comorbidities?

In [18]:
model = smf.ols('walking ~ C(disease_category) + age + bmi + female + educational_level + C(OA_conserv)*C(Diabetes) + C(OA_conserv)*C(Hypertension)', 
                data=maxwalk).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                walking   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     5.058
Date:                Fri, 27 Jun 2025   Prob (F-statistic):           8.77e-08
Time:                        11:07:09   Log-Likelihood:                -3741.4
No. Observations:                1361   AIC:                             7507.
Df Residuals:                    1349   BIC:                             7569.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

##### Logistic Regression: Predicting "High Risk" Patients

In [19]:
maxwalk['low_grip'] = (maxwalk['walking'] < maxwalk['walking'].quantile(0.25)).astype(int)

X = maxwalk[['age', 'bmi', 'female', 'educational_level','OA_conserv', 'Diabetes', 'Hypertension']]
y = maxwalk['low_grip']

logit_model = sm.Logit(y, sm.add_constant(X)).fit()
print(logit_model.summary())


Optimization terminated successfully.
         Current function value: 0.547028
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               low_grip   No. Observations:                 1361
Model:                          Logit   Df Residuals:                     1353
Method:                           MLE   Df Model:                            7
Date:                Fri, 27 Jun 2025   Pseudo R-squ.:                 0.02687
Time:                        11:07:09   Log-Likelihood:                -744.51
converged:                       True   LL-Null:                       -765.06
Covariance Type:            nonrobust   LLR p-value:                 7.691e-07
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 4.6984      1.490      3.153      0.002       1.778       7.619
age     